# Quick illustration of the MOCpy library

In this live version of a jupyter notebook in the browser, some libraries have to be installed in a different way. This is the case for `cds-healpix` that allows manipulation of HEALpix cells and for `MOCpy`. MOC is a hierarchical data format composed of HEALpix cells and time cells (and also of frequency cells, see the develop branch! -- as of 2023-07-05 maybe you're from the future and this is already live?)

## 1. Setup

This two cells can take few tens of seconds to execute, be patient.

In [ ]:
import micropip
# cdshealpix and mocpy install
await micropip.install(["https://rawcdn.githack.com/cds-astro/jupyterlite/cea7bf366dd077953acc578cbb63f2319793a018/content/pyodide/wheels/cdshealpix-0.6.5-cp311-cp311-emscripten_3_1_46_wasm32.whl",
                        "https://rawcdn.githack.com/cds-astro/jupyterlite/cea7bf366dd077953acc578cbb63f2319793a018/content/pyodide/wheels/mocpy-0.13.1-cp311-cp311-emscripten_3_1_46_wasm32.whl"])

In [ ]:
from astropy.coordinates import SkyCoord, Latitude, Longitude
import astropy.units as u
import numpy as np
from mocpy import MOC
import matplotlib.pyplot as plt
from astropy.wcs.utils import skycoord_to_pixel

## 2. Generate a space MOC and random points.

Let's create a way to generate random points in a sqare area of the sky.

In [ ]:
def generate_rand_points(num_points: int, lon_min=-10, lon_max=10, lat_min=-10, lat_max=10):
    """Generate a random number of points in a square region of the sky.

    Parameters
    ----------
    num_points : int
        The number of points to be generated
    lon_min : int, optional
        Minimum longitude, by default -10
    lon_max : int, optional
        Maximum longitude, by default 10
    lat_min : int, optional
        Minimum lattitude, by default -10
    lat_max : int, optional
        Maximum lattitude, by default 10

    Returns
    -------
    (astropy.units.quantity.Quantity, astropy.units.quantity.Quantity)
        Tuple of longitudes and lattitudes
    """
    lon = (np.random.random(num_points) * (lon_max - lon_min) + lon_min) * u.deg
    lat = (np.random.random(num_points) * (lat_max - lat_min) + lat_min) * u.deg
    return lon, lat

And a spatial MOC from a polygon

In [ ]:
lon = Longitude([5, -5, -5, 5], u.deg)
lat = Latitude([5, 5, -5, -5], u.deg)
polygon_moc = MOC.from_polygon(lon, lat)
polygon_moc.display_preview()

You should see a sky projection where the area occupied by our newly created MOC is depicted in red.

## 3. We can now test wether each of the points falls in the square sky-region (the MOC)

In [ ]:
# Generate the points
lon, lat = generate_rand_points(10000)
# Convert to Skycoord object for later use in plot
coordinates = SkyCoord(ra=lon, dec=lat, frame='icrs')
# Calculation of contains_lonlat to the random points
mask = polygon_moc.contains_lonlat(lon, lat)
# Green for True, red for False, and apply that to the mask (might not be the most efficient way to do this)
colors = {True: 'g', False: 'r'}
mask_colors = [colors[bool_contains] for bool_contains in mask]

And plot the results of the `contains` function for 10 000 randomly generated points. 

We'll plot the MOC in transparent blue, the points inside it in green and the points outside in red. 

In [ ]:
fig = plt.figure(figsize=(10, 10))
wcs = polygon_moc.wcs(fig)
x, y = skycoord_to_pixel(coordinates, wcs) # to plot the points on same wcs
ax = fig.add_subplot(projection=wcs)
ax.grid(True)
polygon_moc.fill(ax, wcs, fill=True, color='blue', alpha=0.5)
ax.scatter(x, y, c=mask_colors, alpha=0.5)
polygon_moc.border(ax, wcs, color='blue')

As you can see, it takes more time to plot the result than to do the actual calculation (made in the function `contains_lonlat`)

Feel free to edit this file! Do not worry, it will recover its initial state each time this page is refreshed. A good place to move next is the MOCpy library repository [![MOCpy](https://custom-icon-badges.demolab.com/badge/MOCpy-gray.svg?logo=mocpy&logoWidth=20&)](https://github.com/cds-astro/mocpy "https://github.com/cds-astro/mocpy")